# Generate Learning Objectives for MAE courses with AI

This is a demonstration of the capabilities of generative AI for producing a set of learning objectives for each course of the GW Mechanical and Aerospace Engineering department, based on the brief descriptions listed on the [course bulletin](https://bulletin.gwu.edu/courses/mae/).

Of course, this demo is not intended to suggest that course learning objectives _should_ be auto-generated. It is only meant to showcase the capability, and spark the imagination of readers to the possibilities of highly productive human-AI collaboration in curriculum-development tasks.

# Get the MAE Bulletin and extract course descriptions

Find the text data used in this demo via the short URL: [go.gwu.edu/engcomp1data3](https://go.gwu.edu/engcomp1data3).
To download a file from the internet, we need a function from an external library. Then we grab the file and save it locally. The code needed is:

```python
from urllib.request import urlretrieve
URL = 'https://go.gwu.edu/engcomp1data3'
filename = 'mae_bulletin.txt'
urlretrieve(URL, filename)
```
</br>

Once you have the file with the raw data, the next steps are: 

- _open_ the file "mae_bulletin.txt"
- _read_ the contents into a big string variable
- play with the text to organize it the way we want

In [1]:
mae_bulletin_file = open('mae_bulletin.txt')
mae_bulletin_text = mae_bulletin_file.readlines()
mae_bulletin_text[0:5]

['MAE 1004. Engineering Drawing and Computer Graphics. 0-3 Credits.\n',
 '\n',
 'Introduction to technical drawing, including use of instruments, lettering, geometric construction, sketching, orthographic projection, section view, dimensioning, tolerancing, and pictorial drawing. Introduction to computer graphics, including topics covered in manual drawing and computer-aided drafting.   (Fall and spring).\n',
 '\n',
 'MAE 2117. Engineering Computations. 3 Credits.\n']

We have one big list of lines of text. Next, we want to:

- skip empty lines
- grab the course ID and the course descriptions into new lists

In [2]:
courses = []
descriptions = []

for line in mae_bulletin_text:
    line = line.strip()
    if line == '':
        continue
    elif line.startswith('MAE'):
        courses.append(line)
    else:
        descriptions.append(line)

In [3]:
descriptions[0]

'Introduction to technical drawing, including use of instruments, lettering, geometric construction, sketching, orthographic projection, section view, dimensioning, tolerancing, and pictorial drawing. Introduction to computer graphics, including topics covered in manual drawing and computer-aided drafting.   (Fall and spring).'

Put course ID, title and number of credits into separate lists.

In [4]:
course_id = []
course_title = []
course_credits = []

for course in courses:
    course_info = course.split('. ')
    course_id.append(course_info[0])
    course_title.append(course_info[1])
    course_credits.append(course_info[2])

# Use GPT-3.5 to generate learning objectives based on the descriptions

We will use the Jupyter AI library, with the Open AI provider.

In [5]:
import os
os.environ['OPENAI_API_KEY'] = 'deleted' #DELETE!!!

In [6]:
%load_ext jupyter_ai

In [7]:
system_prompt = "You are an instructional designer, with ample experience working with faculty to improve their course syllabi and content. I will give you brief course descriptions, delimited by three quotes ('''). You will provide a numbered list of between 5 and 7 learning objectives on the basis of the course description, grounded by best practices in backward course design, like using adequate action verbs. Do not give the list a title or heading, and do not explain."
print(system_prompt)

You are an instructional designer, with ample experience working with faculty to improve their course syllabi and content. I will give you brief course descriptions, delimited by three quotes ('''). You will provide a numbered list of between 5 and 7 learning objectives on the basis of the course description, grounded by best practices in backward course design, like using adequate action verbs. Do not give the list a title or heading, and do not explain.


Above, we save a "system prompt" into a string variable. Next, we will try the call to GPT-3.5 using the system prompt and the course title and description for the first item in our course list.

In [8]:
text = course_title[0] + descriptions[0]

In [9]:
%%ai openai-chat:gpt-3.5-turbo
{system_prompt} 
''' {text} '''

- Demonstrate knowledge of technical drawing instruments and their correct usage.
- Apply proper lettering techniques in technical drawings.
- Create accurate geometric constructions using specified methods.
- Generate sketches that effectively communicate design concepts.
- Interpret and create orthographic projections of 3D objects.
- Construct section views of 3D objects based on given specifications.
- Apply appropriate dimensioning and tolerancing principles in technical drawings.
- Utilize computer-aided drafting software to create and modify technical drawings.

Above, GPT generated a set of learning objectives for the first course in our list, using the `%%ai` cell magic. We would like to get objectives for _all_ the courses this way.

To run a cell magic repeatedly in a loop, we can use the `get_ipython().run_cell_magic()` command. The syntax is:

```python
get_ipython().run_cell_magic(magic_name, line, cell)
```
</br>
where:

- `magic_name` is the name of the desired cell magic, without the `%%` prefix. For example, `'timeit`, `'bash'`, etc. This is a required argument.
- `line` is the rest of the first input line as a single string. This is where you can pass any arguments or options to the cell magic. For example, `-n 10 -r 5`, `-l`, etc. This is an optional argument, and it can be left as an empty string (`''`) if no arguments or options are needed.
- `cell` is the body of the cell as a (possibly multiline) string. This is where you write the code that you want to run with the cell magic. For example, `x = 2 + 2`, `echo "Hello world!"`, etc. This is a required argument.

In [10]:
get_ipython().run_cell_magic('ai', 'chatgpt', "{system_prompt}\n ''' {text}'''")

- Identify and use various technical drawing instruments appropriately.
- Demonstrate proficiency in effective lettering techniques for technical drawings.
- Apply geometric construction methods accurately in technical drawings.
- Create sketches that effectively communicate design concepts and ideas.
- Generate orthographic projections and section views of three-dimensional objects accurately.
- Apply dimensioning and tolerancing principles correctly in technical drawings.
- Utilize computer-aided drafting software to create, modify, and enhance technical drawings.

In [11]:
len(descriptions)

108

We have 108 courses to work with, so it would be very tedious to generate learning objectives by hand for each course. We'd like to do it in a loop.

As a test, let's try a very short loop that calls the `%%timeit` cell magic in each iteration:

In [12]:
# This is a loop that runs the %%timeit magic on some code
for i in range(3):
    get_ipython().run_cell_magic('timeit', '', 'x = i + i')

49.9 ns ± 6.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
49.4 ns ± 6.54 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
39 ns ± 6.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Great. Now I'd like to save the output of each call to `get_ipython().run_cell_magic()` into a file. One possible way to save the output of each call into a file is to use the [`%%capture`](http://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-capture) cell magic. We need to give it the name of the variable in which to store output. Since we will call it in a loop, this magic also has to be enclosed in the `get_ipython().run_cell_magic()` command. I will save the third argument in a string variable to make the code more readable.

We could then use the [`%store`](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html) line magic to append the output to a file with the shell redirection `>>`, as follows:

```python
run_it = "get_ipython().run_cell_magic('timeit', '', 'x = i + i')"

# This is a modified loop that saves the output of each call into a file
for i in range(3):
    # Capture the standard output and store it in timeit_output
    get_ipython().run_cell_magic('capture','timeit_output', run_it)
    %store timeit_output.stdout >>timeit.txt # Append the output to timeit.txt
```
</br>

When trying the code above, it created the `timeit.txt` file and saved to it (in append mode) each line resulting from the `%%timeit` command. It also displayed a line for each iteration:
```
Writing 'timeit_output.stdout' (str) to file 'timeit.txt'.`
```
</br>

This may not be desirable if the loop has many iterates (in this case, we want more than 100). It also made me think that we don't just want a file with all the collected outputs: it would be useful to have a list of strings, with index aligned to the course lists `course_title`, etc.

By passing a name to the `%%capture` magic, it will store the captured output in an object of type `<IPython.utils.capture.CapturedIO>` with that name, and with three attributes:

- `stdout`: standard output as a string
- `stderr`: standard error as a string
- `outputs`: a list of rich display outputs

After experimenting with capturing the output of the `%%ai` cell matic, I found that `stdout` was empty and the desired content was in the `outputs` attribute, which was a list of items of type `IPython.utils.capture.RichOutput`. They have a `data` attribute: a dictionary with two key-value pairs, where the content we want is in the value corresponding to the key `'text/markdown'`. Oof! Quite the rabbit hole.
In the end, I settled on this solution (ran for only two iterations while testing).

In [13]:
objectives = []
for i in range(2):
    text = course_title[i] + descriptions[i]
    prompt = f"{system_prompt}\n ''' {text} '''"
    run_it = f"get_ipython().run_cell_magic('ai', 'chatgpt', prompt)"
    get_ipython().run_cell_magic('capture','ai_output', run_it)
    objectives.append(ai_output.outputs[0].data['text/markdown'])

In [14]:
ai_output.outputs[0]

- Understand the concept of round-off errors and discretization errors in numerical computations.
- Apply various methods to solve systems of linear equations.
- Use numerical methods to find roots of equations.
- Apply curve fitting techniques to approximate data.
- Perform numerical Fourier transforms to analyze signals.
- Use numerical differentiation and integration methods to solve engineering problems.
- Apply numerical techniques to solve differential equations.
- Utilize computer applications for engineering computations.

In [15]:
course_title[i]

'Engineering Computations'

In [16]:
type(ai_output.outputs[0])

IPython.utils.capture.RichOutput

In [17]:
ai_output.outputs[0].data

{'text/plain': '<IPython.core.display.Markdown object>',
 'text/markdown': '- Understand the concept of round-off errors and discretization errors in numerical computations.\n- Apply various methods to solve systems of linear equations.\n- Use numerical methods to find roots of equations.\n- Apply curve fitting techniques to approximate data.\n- Perform numerical Fourier transforms to analyze signals.\n- Use numerical differentiation and integration methods to solve engineering problems.\n- Apply numerical techniques to solve differential equations.\n- Utilize computer applications for engineering computations.'}

In [18]:
type(ai_output.outputs[0].data)

dict

In [19]:
type(ai_output.outputs[0].data['text/markdown'])

str

In [20]:
objectives

['- Demonstrate proficiency in using technical drawing instruments for accurate and precise drawings.\n- Apply appropriate lettering techniques to ensure clarity and readability in technical drawings.\n- Use geometric construction methods to create accurate and detailed shapes and structures.\n- Create sketches that effectively convey design concepts and ideas.\n- Generate and interpret orthographic projections and section views of three-dimensional objects.\n- Apply dimensioning and tolerancing principles to ensure proper measurements and specifications in technical drawings.\n- Utilize computer-aided drafting software to create, modify, and enhance technical drawings, incorporating manual drawing techniques.',
 '- Understand the concept of round-off errors and discretization errors in numerical computations.\n- Apply various methods to solve systems of linear equations.\n- Use numerical methods to find roots of equations.\n- Apply curve fitting techniques to approximate data.\n- Perf

In [21]:
len(objectives)

2

Getting to this point was quite a discovery process, but I now have code that can:

- iterate over course titles and descriptions in a loop
- call the `%%ai` cell magic in each loop iteration using the `get_ipython().run_cell_magic()` command to prompt a language model to generate learning objectives
- call the `%%capture` cell magic to capture the output and store it in IO objects
- do some acrobatics to get the content we want appended to a list of strings called `objectives`

**Note:** After a lot of trial-and-error work, I got the following error message:

```
InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4263 tokens. Please reduce the length of the messages.
```
</br>

I didn't realize it, but I was sending the history of prompts and outputs as context to the model, until the context was too big for GPT-3.5-turbo. So I had to reset the chat history with:

```
%%ai openai-chat:gpt-3.5-turbo -r
reset the chat history
```
</br>

All my experimentation to get to this point cost me a total of $0.07 in API usage.

I will reset the chat again, and now I'm confident to run the loop for all the courses. Here we go!

In [22]:
%%ai openai-chat:gpt-3.5-turbo -r
reset the chat history

In [23]:
objectives = []
for i in range(len(descriptions)):
    text = course_title[i] + descriptions[i]
    prompt = f"{system_prompt}\n ''' {text} '''"
    run_it = f"get_ipython().run_cell_magic('ai', 'chatgpt', prompt)"
    get_ipython().run_cell_magic('capture','ai_output', run_it)
    objectives.append(ai_output.outputs[0].data['text/markdown'])
    get_ipython().run_cell_magic('ai','openai-chat:gpt-3.5-turbo -r', 'reset the chat history')

The code block first exited at `i=14`, having again exceeded the maximum context length. I thus added a chat reset within the loop. Running it with the resets to the end took about 30 minutes, and cost a total of about 6 cents in API usage.

In [24]:
i

107

In [25]:
# print any course's generated learning objectives by changing the index 
idx = 6
print(course_title[idx])
print(objectives[idx])

Fluid Mechanics I
- Apply the fundamental concepts of fluid mechanics, including fluid properties, fluid statics, and the integral and differential formulations of conservation of mass, momentum, and energy.
- Analyze and solve fluid mechanics problems using Bernoulli's equation and dimensional analysis.
- Evaluate different types of fluid flow, distinguishing between inviscid and viscous flows.
- Utilize experimental and computational methods in fluid mechanics to analyze and solve real-world problems.
- Demonstrate a deep understanding of the prerequisite knowledge in APSC 2058 to effectively apply it in the context of Fluid Mechanics I.


# Save the generated output for future use

I have aligned Python lists with `course_id`, `course_title` and `objectives`. I would like to create a JSON file with the zipped contents of these lists, meaning, `course_id[i]` should be followed by `course_title[i]` and `objectives[i]` for all values of `i`.

I elected to use JSON rather than pickling the lists so the file is human-readable. 

In [26]:
import json

In [27]:
# Combine the course_id, course_title and objectives lists using zip()
zipped_data = list(zip(course_id, course_title, objectives))

# Convert the zipped data into a JSON file
json_data = json.dumps(zipped_data)

# Write the JSON data to a file
with open('course_objectives.json', 'w') as file:
    file.write(json_data)

In any future session, we can read the JSON file into Python variables with the following code:

```python
import json

# Read the JSON file
with open('data.json', 'r') as file:
    json_data = file.read()

# Parse the JSON data into Python variables
zipped_data = json.loads(json_data)

# Unzip the zipped_data into separate lists
course_id, course_title, objectives = zip(*zipped_data)
```
</br>

When you read the JSON file and want to unzip the data into separate lists, you can use the `zip()` function again with the asterisk `*` to unpack the zipped data.

We can then work with these lists as needed in our subsequent workflow.

# What might we do next?

We have auto-generated learning objectives for 108 courses using GPT-3.5-turbo. No one would use the generated material without human inspection and improvement! But could we use AI for that, too?

Human instructors could manually inspect the generated learning objectives, and flag any that need work. They may compare with historical syllabi, and perhaps mix them with auto-generated ones. Instructors could also identify course descriptions that have grown stale and need updating, after reading the generated course learning objectives.

Let's compare the generated objectives for one course, with the syllabus created by the course instructor—in this case, myself:

In [28]:
zipped_data[1]

('MAE 2117',
 'Engineering Computations',
 '- Apply numerical methods to solve systems of linear equations.\n- Identify and analyze round-off errors and discretization errors in engineering computations.\n- Utilize methods for root finding, curve fitting, and numerical Fourier transform.\n- Implement numerical differentiation and integration techniques.\n- Solve differential equations numerically using appropriate methods.\n- Utilize computer applications for engineering computations.')

My manually created learning objectives are as follows.

> At the end of this course, students will be able to…
>
> 1. manipulate a data series programmatically to organize and explore it, and apply descriptive statistics to it;
> 2. create data visualizations using best practices for communicating with data;
> 3. compute linear regression from data and explain trends and model accuracy;
> 4. explore and visualize both quantitative and categorical data;
 apply full workflows for data analysis, using real data;
> 5. organize computational work, document it, and present it effectively.

What if we prompted the language model to identify the differences? We might then engage in a constructive improvement of our course planning.

Anecdotally, in this particular case, I was intrigued by the learning objectives for my own course, which sounded to me like the old-fashioned numerical course that engineering students usually take.
Ah, yes! I wrote my version of the course for the Spring 2019 semester, when I re-focused it and revised the course description, which I submitted for updating in the university [course bulletin](https://bulletin.gwu.edu/courses/mae/), after faculty approval. The updated description reads:

> Foundations of computational thinking focusing on data practices and computational problem-solving; handling data programmatically, variables and their type, logical operations; reading data from files and cleaning and organizing text data; handling multi-dimensional arrays; basic plotting; linear regression; exploratory data analysis, handling labeled data, and data visualization.

The text data I used here ([posted on GitHub](https://github.com/engineersCode/EngComp1_offtheground/commit/f280fdd71aae2433bccdbe1d8fc4c36013dcf9ce)) is from an _Engineering Computations_ [lesson](http://go.gwu.edu/engcomp1lesson3) on text manipulations with Python, and was scraped from the website in 2017! The old description read: 

In [29]:
descriptions[1]

'Numerical methods for engineering applications. Round-off errors and discretization errors. Methods for solving systems of linear equations, root finding, curve fitting, numerical Fourier transform, and data approximation. Numerical differentiation and integration and numerical solution of differential equations. Computer applications. Prerequisite: MATH 1232. (Fall, Every Year).'

In [30]:
course_id[1]

'MAE 2117'

Not only is the description out-of-date, but it is for a different course! _Engineering Computations_ became a two-course series in 2019, with the first course having the new ID: MAE 1117.

Let's send GPT the updated description, and get new learning objectives.

In [31]:
descriptions[1] = 'Foundations of computational thinking focusing on data practices and computational problem-solving; handling data programmatically, variables and their type, logical operations; reading data from files and cleaning and organizing text data; handling multi-dimensional arrays; basic plotting; linear regression; exploratory data analysis, handling labeled data, and data visualization. (Spring, Every year)'

In [32]:
text = course_title[1] + descriptions[1]

In [33]:
%%ai openai-chat:gpt-3.5-turbo
{system_prompt} 
''' {text} '''

1. Demonstrate understanding of foundational concepts in computational thinking.
2. Apply programming techniques to handle data efficiently and effectively.
3. Analyze and manipulate multi-dimensional arrays to solve computational problems.
4. Implement basic data visualization techniques to effectively communicate findings.
5. Apply exploratory data analysis methods to gain insights from large datasets.
6. Utilize linear regression to analyze and predict patterns in data.
7. Develop skills to read, clean, organize, and process text data from files.

That is indeed closer to the course objectives I originally wrote. Phew!

This final exercise was a small side-track, but still interesting in its own right, I think.

# Some niggling questions:

- Would a human instructor generate learning objectives that are any better than the automatically generated ones?
- Can we imagine a workflow where we begin with automatic generation, and humans add slight improvements?
- How much collective faculty time could be saved by such a workflow?
- Could the generated learning objectives help us identify badly composed (or out-of-date) course descriptions?
- How about using generative AI for the next steps: "Generate a weekly lesson plan to achieve the learning objectives" and "Write an outline for each week's lesson" and more?
- Would such use of AI in any way diminish student learning? After all, **learning is a process, not a destination**, and much less simply a set of contents.

**References**

- Barba, Lorena A. (2019). MAE-1117 Introduction to Engineering Computations (Syllabus). figshare. Online resource. [https://doi.org/10.6084/m9.figshare.7588697.v1](https://doi.org/10.6084/m9.figshare.7588697.v1)
- Barba, Lorena A.; Clementi, Natalia C. (2017). Engineering Computations Module 1: Get data off the ground. figshare. Online resource. [https://doi.org/10.6084/m9.figshare.5673454.v1](https://doi.org/10.6084/m9.figshare.5673454.v1)

In [1]:
from IPython.core.display import HTML
style_file = '../style/custom.css'
HTML(open(style_file, "r").read())